In [1]:
!pip install transformers torchvision kaggle


In [2]:
from google.colab import files
files.upload()  # Upload your kaggle.json file


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sivasubramanianss40","key":"ffdb5dadbe7d15db6b3aa8a41b97afb7"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets download -d sivasubramanianss40/alzheimers-disease-mri-dataset-modified
!unzip alzheimers-disease-mri-dataset-modified.zip -d alzheimers_data


Streaming output truncated to the last 5000 lines.
  inflating: alzheimers_data/Modified DATASET/train/Moderate Impairment/ModerateImpairment (2985).jpg  
  inflating: alzheimers_data/Modified DATASET/train/Moderate Impairment/ModerateImpairment (2986).jpg  
  inflating: alzheimers_data/Modified DATASET/train/Moderate Impairment/ModerateImpairment (2987).jpg  
  inflating: alzheimers_data/Modified DATASET/train/Moderate Impairment/ModerateImpairment (2988).jpg  
  inflating: alzheimers_data/Modified DATASET/train/Moderate Impairment/ModerateImpairment (2989).jpg  
  inflating: alzheimers_data/Modified DATASET/train/Moderate Impairment/ModerateImpairment (2990).jpg  
  inflating: alzheimers_data/Modified DATASET/train/Moderate Impairment/ModerateImpairment (2991).jpg  
  inflating: alzheimers_data/Modified DATASET/train/Moderate Impairment/ModerateImpairment (2992).jpg  
  inflating: alzheimers_data/Modified DATASET/train/Moderate Impairment/ModerateImpairment (2993).jpg  
  inflating: 

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor
from sklearn.model_selection import train_test_split

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load dataset
data_dir = '/content/alzheimers_data/Modified DATASET'
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split dataset
train_data, test_data = train_test_split(dataset, test_size=0.2, stratify=dataset.targets)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [2]:
# Load ViT model and feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
# Load ViT model with mismatched size ignoring
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=4,
    ignore_mismatched_sizes=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [ ]:
model.eval()
correct_predictions = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(pixel_values=images).logits
        preds = torch.argmax(outputs, dim=1)
        correct_predictions += (preds == labels).sum().item()

test_acc = correct_predictions / len(test_loader.dataset)
print(f"Test Accuracy: {test_acc:.4f}")


In [4]:
import torch

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()
epochs = 10  # Train for 10 epochs for better results

for epoch in range(epochs):
    model.train()
    running_loss, correct_predictions = 0.0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Extract features and perform forward pass
        outputs = model(pixel_values=images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct_predictions += (preds == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct_predictions / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")


NameError: name 'model' is not defined

In [ ]:
# Import necessary libraries
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor
from sklearn.model_selection import train_test_split

# Image transformations (resize from 128x128 to 224x224)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize from 128x128 to 224x224
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalization
])

# Load dataset
data_dir = '/content/alzheimers_data/Modified DATASET'
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split dataset
train_data, test_data = train_test_split(dataset, test_size=0.2, stratify=dataset.targets)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Load ViT model
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=4,  # Assuming 4 classes for Alzheimer's stages
    ignore_mismatched_sizes=True
)

# Ensure model is on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Now define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)  # Optimizer
criterion = torch.nn.CrossEntropyLoss()  # Loss function

# Training loop
epochs = 10  # Number of training epochs
for epoch in range(epochs):
    model.train()
    running_loss, correct_predictions = 0.0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(pixel_values=images).logits  # Forward pass
        loss = criterion(outputs, labels)  # Calculate loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        running_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct_predictions += (preds == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct_predictions / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to

In [ ]:
torch.save(model.state_dict(), "vit_alzheimers_model.pth")
print("Model saved successfully!")
